## Web Scraping Automatizado: IEEE Xplore

Este notebook implementa un **bot de web scraping automatizado** para descargar artículos científicos en formato BibTeX desde IEEE Xplore.

### Objetivos:
1. **Autenticación automática** vía proxy institucional
2. **Navegación programática** a IEEE Xplore
3. **Búsqueda automatizada** de términos específicos
4. **Descarga masiva** de archivos BibTeX con paginación
5. **Organización** de archivos descargados

### Flujo del Proceso:
```
Login Proxy → Autenticación Google → Selección Facultad → 
Acceso IEEE → Búsqueda → Configuración Resultados → 
Loop de Paginación → Descarga BibTeX → Renombrado → Siguiente Página
```

## Paso 1 — Imports y carga de variables de entorno

## Este bloque importa las librerías necesarias para manejar datos, controlar el navegador con Selenium y cargar variables desde un archivo .env.

In [1]:
#imports requeridos para que el proyecto funcione
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

# Obtener y validar variables de entorno
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

# Validación crítica de variables
if not email or not password:
    raise ValueError("ERROR: EMAIL y PASSWORD deben estar definidos en el archivo .env")

print(f"Variables cargadas correctamente")
print(f"Email: {email[:3]}***@{email.split('@')[1] if '@' in email else '***'}")

Variables cargadas correctamente
Email: bra***@uqvirtual.edu.co


## Paso 2 — Configuración de ruta y opciones de descarga en Chrome

## Este bloque define la carpeta donde se guardarán las descargas y configura las preferencias del navegador Chrome para automatizar ese proceso.

In [2]:
# Configurar la ruta de descarga
download_path = os.getenv("DOWNLOAD_PATH")

# Validación crítica de ruta de descarga
if not download_path:
    raise ValueError("ERROR: DOWNLOAD_PATH debe estar definido en el archivo .env")

os.makedirs(download_path, exist_ok=True)

chrome_options = webdriver.ChromeOptions()

# Configuración de preferencias de descargas
prefs = {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
chrome_options.add_experimental_option("prefs", prefs)

# Argumentos adicionales para estabilidad
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-download-notification")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

print(f"Opciones de Chrome configuradas")
print(f"Ruta de descarga: {download_path}")

Opciones de Chrome configuradas
Ruta de descarga: /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas


## Paso 3 — Inicialización del navegador y apertura de la página

## Este bloque crea una instancia del navegador Chrome con las opciones configuradas previamente y accede a la URL deseada.

In [3]:
# Iniciar el navegador con protección
driver = None

try:
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    print("Navegador Chrome iniciado correctamente")
    
    # Abrir la URL
    url1 = 'https://login.intelproxy.com/v2/inicio?cuenta=7Ah6RNpGWF22jjyq'
    driver.get(url1)
    print(f"Navegando a: {url1}")
    
except Exception as e:
    print(f"ERROR: Error al iniciar el navegador: {e}")
    if driver:
        driver.quit()
    raise

Navegador Chrome iniciado correctamente
Navegando a: https://login.intelproxy.com/v2/inicio?cuenta=7Ah6RNpGWF22jjyq


## Paso 4 — Esperar y hacer clic en el botón de inicio de sesión

## Este bloque espera hasta que el botón de inicio de sesión esté disponible y realiza el clic para continuar con el proceso.

In [4]:
login_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[2]/a"))
)
login_button.click()

## Paso 5 — Ingresar el correo electrónico

## Este bloque espera el campo de correo, introduce el valor obtenido del archivo .env y envía la tecla Enter para continuar.

In [5]:
email_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "identifierId"))
)

email_input.send_keys(email)
email_input.send_keys(Keys.RETURN)

## Paso 6 — Ingresar la contraseña

## Este bloque espera el campo de contraseña, escribe la clave obtenida del archivo .env y envía la tecla Enter para iniciar sesión.

In [6]:
password_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.NAME, "Passwd"))
)# Nombre del campo de contraseña
password_input.send_keys(password)
password_input.send_keys(Keys.RETURN)

## Paso 7 — Acceder al apartado "Fac. Ingeniería"

## Este bloque localiza y hace clic en el botón que despliega la sección correspondiente a “Fac. Ingeniería”.

In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Buscar el botón <summary> que contiene "Fac. Ingeniería"
boton = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//summary[contains(., 'Fac. Ingeniería')]"))
)

time.sleep(2)
boton.click()

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
#0 0x55a1dfbd8aea <unknown>
#1 0x55a1df624cdb <unknown>
#2 0x55a1df60b6ae <unknown>
#3 0x55a1df6339ca <unknown>
#4 0x55a1df6a5cef <unknown>
#5 0x55a1df6c36a5 <unknown>
#6 0x55a1df669aca <unknown>
#7 0x55a1df66a7d1 <unknown>
#8 0x55a1dfb9fab9 <unknown>
#9 0x55a1dfba2a8c <unknown>
#10 0x55a1dfb88d49 <unknown>
#11 0x55a1dfba3685 <unknown>
#12 0x55a1dfb706c3 <unknown>
#13 0x55a1dfbc57d8 <unknown>
#14 0x55a1dfbc59b3 <unknown>
#15 0x55a1dfbd7a83 <unknown>
#16 0x7fc065d68464 start_thread
#17 0x7fc065deb5ac __clone3


## Paso 8 — Localizar el enlace a IEEE

## Este bloque espera a que el enlace correspondiente a “IEEE” esté disponible para su interacción en la página.

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Buscar el enlace a IEEE por su XPath correcto
enlaceIEEE = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='facingenieraieeeinstituteofelectricalandelectronicsengineersdescubridor']//h2[@class='result-title']/a"))
)


# Paso 9 — Hacer clic en el enlace de IEEE

# Este bloque realiza la acción de clic en el enlace previamente localizado para acceder al contenido o recurso correspondiente.

In [ ]:
# Hacer clic en el enlace
enlaceIEEE.click()

TimeoutException: Message: timeout: Timed out receiving message from renderer: 30.000
  (Session info: chrome=142.0.7444.59)
Stacktrace:
#0 0x561277764aea <unknown>
#1 0x5612771b0cdb <unknown>
#2 0x561277198c84 <unknown>
#3 0x561277198967 <unknown>
#4 0x56127719676f <unknown>
#5 0x5612771970ef <unknown>
#6 0x5612771a5b33 <unknown>
#7 0x5612771bf491 <unknown>
#8 0x5612771c5cab <unknown>
#9 0x56127719782d <unknown>
#10 0x5612771bf239 <unknown>
#11 0x56127724ffff <unknown>
#12 0x5612771f5aca <unknown>
#13 0x5612771f67d1 <unknown>
#14 0x56127772bab9 <unknown>
#15 0x56127772ea8c <unknown>
#16 0x561277714d49 <unknown>
#17 0x56127772f685 <unknown>
#18 0x5612776fc6c3 <unknown>
#19 0x5612777517d8 <unknown>
#20 0x5612777519b3 <unknown>
#21 0x561277763a83 <unknown>
#22 0x7f32832a4464 start_thread
#23 0x7f32833275ac __clone3


## Paso 10 — Cerrar el banner de cookies

## Este bloque intenta detectar y cerrar el aviso de cookies que puede aparecer en la página antes de continuar con la automatización.

In [ ]:
try:
    # Esperar a que aparezca cualquiera de los botones
    boton_cookies = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.osano-cm-dialog__close, button.osano-cm-denyAll, button.osano-cm-accept-all"))
    )
    driver.execute_script("arguments[0].click();", boton_cookies)
    print("🍪 Banner de cookies cerrado.")
except Exception as e:
    print(f"⚠ No apareció el banner de cookies o ya estaba cerrado: {e}")


🍪 Banner de cookies cerrado.


## Paso 11 — Activar la barra de búsqueda en la página de IEEE

## Este bloque espera a que la barra de búsqueda esté disponible, la desplaza a la vista y realiza un clic para activarla.

In [ ]:
# Esperar a que la barra de búsqueda esté clickeable y visible
barra_busqueda = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='LayoutWrapper']/div/div/div[3]/div/xpl-root/header/xpl-header/div/div[2]/div[2]/xpl-search-bar-migr/div/form/div[2]/div/div[1]/xpl-typeahead-migr/div/input"))
)

# Hacer scroll al elemento para asegurarse de que esté visible
driver.execute_script("arguments[0].scrollIntoView(true);", barra_busqueda)
time.sleep(1)

# Hacer clic primero para asegurarse de que está activo
barra_busqueda.click()
time.sleep(0.5)

# Paso 12 — Ingresar el texto de búsqueda

# Este bloque limpia el campo de búsqueda y escribe el término deseado para realizar la consulta.

In [ ]:
# Escribir "generative artificial intelligence" entre comillas para reducir el rango de busqueda
barra_busqueda.clear()  # Limpiar cualquier texto previo
barra_busqueda.send_keys('generative artificial intelligence')
time.sleep(1)  # Pequeña pausa para asegurar que el texto se escribió

# Paso 13 — Esperar el botón de búsqueda

# Este bloque localiza y espera a que el botón de búsqueda esté disponible para su uso.

In [ ]:
# Esperar a que el botón de búsqueda esté clickeable
boton_buscar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='LayoutWrapper']/div/div/div[3]/div/xpl-root/header/xpl-header/div/div[2]/div[2]/xpl-search-bar-migr/div/form/div[2]/div/div[2]/button"))
)

# Paso 14 — Ejecutar la búsqueda

# Este bloque realiza el clic en el botón de búsqueda para iniciar la consulta en la página.

In [ ]:
boton_buscar.click()

# Paso 15 — Verificar la finalización de la descarga

# Este bloque define una función que espera la aparición de un nuevo archivo .bib en la carpeta de descargas, confirmando que la descarga se completó correctamente.

In [ ]:
# Función para verificar que la descarga se completó
def esperar_descarga(carpeta, tiempo_max=30):
    """
    Espera hasta que aparezca un nuevo archivo .bib en la carpeta de descargas
    o hasta que se agote el tiempo máximo.
    """
    inicio = time.time()
    archivos_iniciales = set([f for f in os.listdir(carpeta) if f.endswith('.bib')])
    
    while time.time() - inicio < tiempo_max:
        time.sleep(2)  # Comprobar cada 2 segundos
        archivos_actuales = set([f for f in os.listdir(carpeta) if f.endswith('.bib')])
        nuevos_archivos = archivos_actuales - archivos_iniciales
        
        if nuevos_archivos:
            # Encontró nuevo archivo
            nuevo_archivo = list(nuevos_archivos)[0]
            return os.path.join(carpeta, nuevo_archivo)
        
    print("⚠️ Tiempo de espera de descarga agotado.")
    return None


# Paso 16 — Cerrar ventana emergente de cookies

# ste bloque intenta localizar y cerrar una ventana de cookies adicional que puede aparecer al cargar la página.

In [ ]:
try:
    # Esperar a que aparezca el botón de aceptar/cerrar cookies
    boton_cookies = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[2]/button[2]"))  # Usa el XPath correcto
    )
    boton_cookies.click()
    print("Se cerró la ventana de cookies.")
except:
    print("⚠ No apareció la ventana de cookies o ya estaba cerrada.")

⚠ No apareció la ventana de cookies o ya estaba cerrada.


# Paso 17 — Mostrar más resultados por página

# Este bloque aumenta la cantidad de resultados visibles en la página, facilitando el acceso y descarga de más elementos sin cambiar de página.

In [ ]:
# 1. Hacer clic en el menú "Items Per Page"
itemsPerPage_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'dropdownPerPageLabel'))
)
itemsPerPage_link.click()

# 2. Seleccionar la opción deseada (ejemplo: 100)
option_100 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '100')]"))
)
option_100.click()


# Paso 18 — Descarga automatizada de resultados en formato .bib

# Este bloque implementa la automatización completa para descargar los resultados de búsqueda de IEEE Xplore en formato BibTeX, página por página, guardándolos con nombres personalizados.

In [ ]:
import os
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

# Función auxiliar para esperar descargas nuevas
def esperar_descarga(download_path, archivos_antes, timeout=30):
    """
    Espera a que aparezca un nuevo archivo .bib en la carpeta de descargas
    que no esté en la lista anterior.
    """
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            archivos_actuales = set(f for f in os.listdir(download_path) if f.endswith(".bib"))
            nuevos = archivos_actuales - archivos_antes
            if nuevos:
                nuevo_archivo = max([os.path.join(download_path, f) for f in nuevos],
                                  key=os.path.getctime)
                # Verificar que tenga contenido
                if os.path.getsize(nuevo_archivo) > 0:
                    return nuevo_archivo
        except Exception as e:
            print(f"ADVERTENCIA: Error al verificar descargas: {e}")
        time.sleep(1)
    return None

# Configura aquí tu carpeta de descargas base
base_download_path = os.getenv("DOWNLOAD_PATH")

# Crear carpeta ieee dentro de descargas
ieee_folder = os.path.join(base_download_path, "ieee")
if not os.path.exists(ieee_folder):
    os.makedirs(ieee_folder)
    print(f"Carpeta creada: {ieee_folder}")
else:
    print(f"Usando carpeta existente: {ieee_folder}")

# Usar la carpeta ieee como directorio de trabajo
download_path = ieee_folder

# Inicializar el contador de páginas y errores consecutivos
page_num = 1
hay_mas_paginas = True
errores_consecutivos = 0
MAX_ERRORES_CONSECUTIVOS = 3

print(f"\n{'='*60}")
print(f"INICIANDO DESCARGA AUTOMATIZADA DE IEEE XPLORE")
print(f"{'='*60}\n")

# Bucle para navegar a través de todas las páginas
while hay_mas_paginas:
    print(f"\n{'─'*60}")
    print(f"Procesando página {page_num}")
    print(f"{'─'*60}")

    try:
        # Espera inteligente para carga de página
        WebDriverWait(driver, 15).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
        time.sleep(2)

        # Seleccionar checkbox con manejo de errores
        try:
            checkbox = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH,
                    '//*[@id="xplMainContent"]/div[2]/div[2]/xpl-results-list/div[2]/label/input'))
            )
            if not checkbox.is_selected():
                driver.execute_script("arguments[0].click();", checkbox)
            time.sleep(1)
            print("Checkbox seleccionado")
            errores_consecutivos = 0  # Resetear contador
        except TimeoutException:
            print(f"ADVERTENCIA: No se pudo seleccionar checkbox en página {page_num}")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                print(f"ERROR: Demasiados errores consecutivos. Deteniendo proceso.")
                break
            continue

        # Botón Export con manejo de errores
        try:
            enlace = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH,
                    "//*[@id='xplMainContent']/div[1]/div[1]/ul/li[3]/xpl-export-search-results/button"))
            )
            driver.execute_script("arguments[0].click();", enlace)
            time.sleep(2)
            print("Menú de exportación abierto")
        except TimeoutException:
            print(f"ADVERTENCIA: No se pudo abrir menú de exportación en página {page_num}")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
            continue

        # Opción Citations con manejo de errores
        try:
            all_results = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH,
                    "/html/body/ngb-modal-window/div/div/div[1]/ul/li[2]/a"))
            )
            all_results.click()
            time.sleep(2)
            print("Opción 'Citations' seleccionada")
        except TimeoutException:
            print(f"ADVERTENCIA: No se pudo seleccionar Citations en página {page_num}")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
            continue

        # Seleccionar BibTeX con manejo de errores
        try:
            bibtex_link = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH,
                    "/html/body/ngb-modal-window/div/div/div[2]/div/xpl-citation-download/form/div[1]/section[1]/div/label[2]/input"))
            )
            bibtex_link.click()
            time.sleep(1)
            print("Formato BibTeX seleccionado")
        except TimeoutException:
            print(f"ADVERTENCIA: No se pudo seleccionar BibTeX en página {page_num}")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
            continue

        # Citation + Abstract (no crítico)
        try:
            citAndAbstract_link = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH,
                    "/html/body/ngb-modal-window/div/div/div[2]/div/xpl-citation-download/form/div[1]/section[2]/div/label[2]/input"))
            )
            citAndAbstract_link.click()
            time.sleep(1)
            print("Opción 'Citation and Abstract' seleccionada")
        except TimeoutException:
            print(f"ADVERTENCIA: No se pudo seleccionar Citation and Abstract")
            # No es crítico, continuar

        # Guardar estado de descargas antes de bajar
        archivos_antes_base = set([f for f in os.listdir(base_download_path) if f.endswith(".bib")])

        # Botón descargar con manejo de errores
        try:
            download_button = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH,
                    "/html/body/ngb-modal-window/div/div/div[2]/div/xpl-citation-download/form/div[2]/button[2]"))
            )
            download_button.click()
            print(f"Iniciando descarga para página {page_num}...")

            # Esperar el archivo en la carpeta base de descargas
            nuevo_archivo = None
            end_time = time.time() + 50
            
            while time.time() < end_time:
                try:
                    archivos_actuales = set(f for f in os.listdir(base_download_path) if f.endswith(".bib"))
                    nuevos = archivos_actuales - archivos_antes_base
                    
                    if nuevos:
                        archivo_descargado = max([os.path.join(base_download_path, f) for f in nuevos],
                                          key=os.path.getctime)
                        
                        # Verificar que tenga contenido
                        if os.path.getsize(archivo_descargado) > 0:
                            # Mover a la carpeta ieee
                            nuevo_nombre = f"ieee_generative_ai_page_{page_num}.bib"
                            ruta_nueva = os.path.join(download_path, nuevo_nombre)
                            
                            # Si ya existe, eliminarlo
                            if os.path.exists(ruta_nueva):
                                os.remove(ruta_nueva)
                            
                            os.rename(archivo_descargado, ruta_nueva)
                            print(f"Archivo guardado como: {nuevo_nombre}")
                            nuevo_archivo = ruta_nueva
                            errores_consecutivos = 0  # Resetear contador
                            break
                except Exception as e:
                    print(f"ADVERTENCIA: Error al verificar descarga: {e}")
                
                time.sleep(1)
            
            if not nuevo_archivo:
                print(f"ERROR: No se detectó descarga en página {page_num}")
                errores_consecutivos += 1
                if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                    break

        except TimeoutException:
            print(f"ADVERTENCIA: Timeout en el proceso de descarga en página {page_num}")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
        except Exception as e:
            print(f"ADVERTENCIA: Error inesperado en descarga: {type(e).__name__}: {e}")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break

        # Cerrar modal
        try:
            cerrarVentana_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH,
                    "/html/body/ngb-modal-window/div/div/div[1]/div/i"))
            )
            cerrarVentana_link.click()
            time.sleep(1)
        except:
            # Intentar cerrar con ESC
            ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            time.sleep(1)

        # Verificar si hay más páginas
        try:
            siguiente_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 
                    'button[class^="stats-Pagination_arrow_next_"], '
                    'button[aria-label="Next page of search results"], '
                    'button.next, a.next, li.next-btn button'))
            )
            
            # Verificar que el botón no esté deshabilitado
            if siguiente_btn.is_enabled():
                driver.execute_script("arguments[0].click();", siguiente_btn)
                page_num += 1
                print(f"Avanzando a la página {page_num}...")
                time.sleep(4)
            else:
                print("INFO: Botón de siguiente página deshabilitado. Fin del recorrido.")
                hay_mas_paginas = False
            
        except TimeoutException:
            print("INFO: No se encontró botón de siguiente página. Fin del recorrido.")
            hay_mas_paginas = False
        except Exception as e:
            print(f"ADVERTENCIA: Error al intentar avanzar de página: {type(e).__name__}")
            hay_mas_paginas = False

    except KeyboardInterrupt:
        print("\nADVERTENCIA: Proceso interrumpido por el usuario")
        break
    except Exception as e:
        print(f"ERROR: Error inesperado en página {page_num}: {type(e).__name__}: {e}")
        errores_consecutivos += 1
        if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
            print(f"ERROR: Demasiados errores consecutivos. Deteniendo proceso.")
            break
        continue

print(f"\n{'='*60}")
print(f"Proceso completado")
print(f"Total de páginas procesadas: {page_num}")
print(f"Archivos guardados en: {download_path}")
print(f"{'='*60}\n")

📁 Usando carpeta existente: /home/yep/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee

Procesando página 1...
⬇️ Intentando descarga para página 1...
✅ Archivo guardado como: ieee_generative_ai_page_1.bib
➡️ Avanzando a la página 2...

Procesando página 2...
⬇️ Intentando descarga para página 2...
✅ Archivo guardado como: ieee_generative_ai_page_2.bib
➡️ Avanzando a la página 3...

Procesando página 3...
⬇️ Intentando descarga para página 3...
✅ Archivo guardado como: ieee_generative_ai_page_3.bib
➡️ Avanzando a la página 4...

Procesando página 4...
⬇️ Intentando descarga para página 4...
✅ Archivo guardado como: ieee_generative_ai_page_4.bib
➡️ Avanzando a la página 5...

Procesando página 5...
⬇️ Intentando descarga para página 5...
✅ Archivo guardado como: ieee_generative_ai_page_5.bib
➡️ Avanzando a la página 6...

Procesando página 6...
⬇️ Intentando descarga para página 6...
✅ Archivo guardado como: ieee_generative_ai_page_6.bib
➡️ Avanzando a la página 7...

Procesando pág

KeyboardInterrupt: 

In [ ]:
# Cerrar el navegador de forma segura
if driver:
    try:
        print("Cerrando navegador...")
        driver.quit()
        print("Navegador cerrado correctamente")
    except Exception as e:
        print(f"ADVERTENCIA: Error al cerrar navegador: {e}")
else:
    print("INFO: No hay navegador activo para cerrar")